This notebook verifies math in Appendix A. Perspective effect in Oh & Evans 2020.

In [13]:
from sympy import symbols, simplify, latex
from sympy import cos, sin, Matrix, diff, N
import numpy as np

ra, dec = symbols('alpha, delta')
vra,vdec,vr = symbols(r'v_\alpha, v_\delta, v_r')
vx,vy,vz = symbols('v_x v_y v_z')
delta_ra, delta_dec= symbols(r'\Delta\alpha \Delta\delta')

R = Matrix([
    [-sin(ra), cos(ra), 0.],
    [-sin(dec)*cos(ra), -sin(dec)*sin(ra), cos(dec)],
    [cos(dec)*cos(ra), cos(dec)*sin(ra), sin(dec)]
])

In [2]:
R

Matrix([
[           -sin(alpha),             cos(alpha),        0.0],
[-sin(delta)*cos(alpha), -sin(alpha)*sin(delta), cos(delta)],
[ cos(alpha)*cos(delta),  sin(alpha)*cos(delta), sin(delta)]])

In [3]:
simplify(R.inv()) == R.T

True

In [4]:
diff(R, ra)

Matrix([
[           -cos(alpha),            -sin(alpha), 0],
[ sin(alpha)*sin(delta), -sin(delta)*cos(alpha), 0],
[-sin(alpha)*cos(delta),  cos(alpha)*cos(delta), 0]])

In [5]:
diff(R, dec)

Matrix([
[                     0,                      0,           0],
[-cos(alpha)*cos(delta), -sin(alpha)*cos(delta), -sin(delta)],
[-sin(delta)*cos(alpha), -sin(alpha)*sin(delta),  cos(delta)]])

## Geneneral $\Delta v_\mathrm{sphere}$ to the first order

In [14]:
vvec = Matrix([
    [vx],
    [vy],
    [vz]
])

In [15]:
delta_v_sphere = diff(R, ra)*vvec*delta_ra + diff(R, dec)*vvec*delta_dec
delta_v_sphere

Matrix([
[                                                                                                               \Delta\alpha*(-v_x*cos(alpha) - v_y*sin(alpha))],
[ \Delta\alpha*(v_x*sin(alpha)*sin(delta) - v_y*sin(delta)*cos(alpha)) + \Delta\delta*(-v_x*cos(alpha)*cos(delta) - v_y*sin(alpha)*cos(delta) - v_z*sin(delta))],
[\Delta\alpha*(-v_x*sin(alpha)*cos(delta) + v_y*cos(alpha)*cos(delta)) + \Delta\delta*(-v_x*sin(delta)*cos(alpha) - v_y*sin(alpha)*sin(delta) + v_z*cos(delta))]])

We can express this with $v_\mathrm{sphere} = [v_\alpha,\,v_\delta,\,v_r]^T$ **at** $(\alpha,\,\delta)$.

In [16]:
vvec = R.T @ Matrix([[vra],[vdec],[vr]])
delta_v_sphere = diff(R, ra)*vvec*delta_ra + diff(R, dec)*vvec*delta_dec

In [17]:
simplify(delta_v_sphere)

Matrix([
[     \Delta\alpha*(v_\delta*sin(delta) - v_r*cos(delta))],
[    -\Delta\alpha*v_\alpha*sin(delta) - \Delta\delta*v_r],
[\Delta\alpha*v_\alpha*cos(delta) + \Delta\delta*v_\delta]])

## A special case: $\vec{v}_0$ is radial.

In [56]:
v_radial = Matrix([
    [0],
    [0],
    [vr]
])

v0 = R.T * v_radial

In [57]:
dMdrav0 = simplify(diff(R, ra) * v0)
dMddecv0 = simplify(diff(R, dec)*v0)

dMdrav0*delta_ra + dMddecv0*delta_dec

Matrix([
[-\Delta\alpha*v_r*cos(delta)],
[           -\Delta\delta*v_r],
[                           0]])

### second-order terms

In [58]:
delta_v_sphere2 = simplify(diff(R, ra, 2) *v0 * delta_ra**2  + diff(R, dec, 2) *v0 * delta_dec**2 + 2*diff(R, ra,dec) * v0 * delta_ra * delta_dec)

In [60]:
delta_v_sphere2.subs({ra:np.deg2rad(45), dec:np.deg2rad(45)})

Matrix([
[                                           0],
[                     0.5*\Delta\alpha**2*v_r],
[-v_r*(0.5*\Delta\alpha**2 + \Delta\delta**2)]])

In [61]:
delta_v_sphere2.subs({ra:np.deg2rad(135), dec:np.deg2rad(135)})

Matrix([
[                                           0],
[                    -0.5*\Delta\alpha**2*v_r],
[-v_r*(0.5*\Delta\alpha**2 + \Delta\delta**2)]])

In [37]:
N(delta_v_sphere.subs({ra:np.deg2rad(45),dec:np.deg2rad(45), vx:5.0, vy:5.0, vz:7.07106781}), 3)

Matrix([
[                          -7.07*\Delta\alpha],
[  -4.44e-16*\Delta\alpha - 10.0*\Delta\delta],
[4.44e-16*\Delta\alpha - 1.32e-9*\Delta\delta]])

In [38]:
N(delta_v_sphere.subs({ra:np.deg2rad(300),dec:np.deg2rad(45), vx:5.0, vy:5.0, vz:7.07106781}), 3)

Matrix([
[                     1.83*\Delta\alpha],
[-4.83*\Delta\alpha - 3.71*\Delta\delta],
[ 4.83*\Delta\alpha + 6.29*\Delta\delta]])

In [39]:
N(delta_v_sphere.subs({ra:np.deg2rad(340),dec:np.deg2rad(-65), vx:5.0, vy:5.0, vz:7.07106781}), 3)

Matrix([
[                   -2.99*\Delta\alpha],
[5.81*\Delta\alpha + 5.15*\Delta\delta],
[ 2.71*\Delta\alpha + 5.7*\Delta\delta]])